In [ ]:
!pip install unsloth

!pip install -q -U bitsandbytes==0.45.3
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 8096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import pandas as pd

test_df = pd.read_csv('test_df').drop('Unnamed: 0', axis=1)
train_df = pd.read_csv('train_df').drop('Unnamed: 0', axis=1)
eval_df = pd.read_csv('eval_df').drop('Unnamed: 0', axis=1)


In [3]:
eval_df = eval_df.sample(frac=1, random_state=42)
train_df = train_df.sample(frac=1, random_state=42)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
prompt_format = """Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим контекст. 
Напиши ответ, который соответствующим образом отвечает на вопрос.

### Instruction:
Ты юридический консультант. Тебе дан контекст ситуации, в которой ты должен хорошо разобраться 
и дать короткий, но содержательный ответ на заданный вопрос.

### Input:
Контекст: {}

Вопрос: {}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    contexts   = examples["Контекст"]
    questions  = examples["Вопрос"]
    outputs    = examples["Ответ"]
    texts      = []
    for context, question, output in zip(contexts, questions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_format.format(context, question, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
train_df = train_df.reset_index().drop('index', axis=1)
eval_df = eval_df.reset_index().drop('index', axis=1)
train_df

,Контекст,Вопрос,Ответ
0,Истец является собственником комнаты в коммуна...,Какие доказательства мне нужны для подтвержден...,Необходимы: \n- Заключение независимой оценки...
1,Пешеход (ФИО1) был оштрафован на 500 рублей за...,"Можно ли избежать штрафа, если пешеходный пере...","Нет, п. 4.3 ПДД обязывает пешеходов переходить..."
2,Страховая компания СПАО «Ингосстрах» выплатила...,"Что делать, если страховая компания утверждает...",Требуйте от страховщика доказательств вашего у...
3,Водитель (истец) совершил наезд на стоящий авт...,"Можно ли избежать лишения прав, если поврежден...","Возможно, но малозначительность (ст. 2.9 КоАП ..."
4,ООО «Юрэнергоконсалт» было привлечено к админи...,Можно ли оспорить штраф за непредставление инф...,"Да, можно. Если предписание, на котором основа..."
...,...,...,...
11323,Юридическое лицо ООО «Краском» было привлечено...,Может ли неисполнение предписания органа пожар...,"Да, если нарушение требований пожарной безопас..."
11324,Директор ООО «Атлант» был привлечен к админист...,"Можно ли избежать штрафа, если правонарушение ...","Да, возможно. Согласно ч. 1 ст. 4.1.1 КоАП РФ,..."
11325,Дорожно-транспортное происшествие произошло ме...,"Кто несет ответственность за вред, если автомо...",По ст. 1079 ГК РФ ответственность несет владел...
11326,В мае 2023 года произошло ДТП по вине водителя...,Можно ли взыскать неустойку за просрочку страх...,"Да, по п. 21 ст. 12 Закона об ОСАГО неустойка ..."


In [ ]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
eval_ds = Dataset.from_pandas(eval_df)
test_ds = Dataset.from_pandas(test_df)

In [ ]:
train_ds = train_ds.map(formatting_prompts_func, batched = True)
eval_ds = eval_ds.map(formatting_prompts_func, batched = True)
test_ds = test_ds.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/11328 [00:00<?, ? examples/s]

Map: 100%|██████████| 300/300 [00:00<00:00, 39893.83 examples/s]


In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

BATCH_SIZE = 256
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 2e-4
TRAIN_STEPS = 500
TRAIN_EPOCHS = 3
OUTPUT_DIR = "./QLoRA_64_2_3"

training_arguments = TrainingArguments(
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
    warmup_steps = 50,
    num_train_epochs = TRAIN_EPOCHS, # Set this for 1 full training run.
    #max_steps = TRAIN_STEPS,
    learning_rate = LEARNING_RATE,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    eval_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 1,
    save_steps = 1,
    save_total_limit = 3,
    load_best_model_at_end = True,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = OUTPUT_DIR,
    report_to = "wandb" #"tensorboard", # Use this for WandB etc
    # overwrite_output_dir=True
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = eval_ds,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_arguments,
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 297/297 [00:01<00:00, 235.45 examples/s]


In [ ]:
trainer_stats = trainer.train()

In [ ]:
from huggingface_hub import login
login(hf_token)

In [ ]:
model.save_pretrained("./qlora_64_2_3") 
tokenizer.save_pretrained("./qlora_64_2_3")

In [ ]:
model.push_to_hub("foxxar04/llama_qlora")
tokenizer.push_to_hub("foxxar04/llama_qlora")